In [1]:
!pip install mne mne-bids osl-ephys

  Using cached tabulate-0.9.0-py3-none-any.whl.metadata (34 kB)
  Using cached dill-0.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached lxml-6.0.2-cp312-cp312-win_amd64.whl.metadata (3.7 kB)
  Using cached markdown_it_py-4.0.0-py3-none-any.whl.metadata (7.3 kB)
  Using cached narwhals-2.11.0-py3-none-any.whl.metadata (11 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/17.7 MB ? eta -:--:--
   - -------------------------------------- 0.8/17.7 MB 6.7 MB/s eta 0:00:03
   -- ------------------------------------- 1.0/17.7 MB 5.0 MB/s eta 0:00:04
   -- ------------------------------------- 1.0/17.7 MB 5.0 MB/s eta 0:00:04
   -- ------------------------------------- 1.0/17.7 MB 5.0 MB/s eta 0:00:04
   -- ------------------------------------- 1.0/17.7 MB 5.0 MB/s eta 0:00:04
   -- ------------------------------------- 1.0/17.7 MB 5.0 MB/s eta 0:00:04
   ---- ----------------------------------- 2.1/17.7 MB 1.5 MB/s eta 0:00:11


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
# -*- coding: utf-8 -*-
"""
KỊCH BẢN TIỀN XỬ LÝ (PREPROCESSING) EEG CHO PROJECT ODDBALL

Mục tiêu:
1. Tải dữ liệu BIDS đã sắp xếp.
2. Áp dụng lọc (Filter) và lọc nhiễu điện (Notch).
3. Chạy ICA để tìm và loại bỏ nhiễu chớp mắt (EOG).
4. Trực quan hóa để kiểm tra và đánh giá (an toàn, không lỗi).
5. Lưu dữ liệu sạch ra tệp .fif trong thư mục 'derivatives'.
"""

import os
import mne
from mne_bids import BIDSPath, read_raw_bids, get_entity_vals
import matplotlib.pyplot as plt

# =============================================================================
# BƯỚC 0: THIẾT LẬP CÁC THÔNG SỐ VÀ ĐƯỜNG DẪN
# =============================================================================
project_dir = r'D:\EEG' 

# Tự động tìm tất cả các subject có trong thư mục
try:
    SUBJECT_IDS = sorted(get_entity_vals(project_dir, 'subject'))
    if not SUBJECT_IDS:
        raise FileNotFoundError
except FileNotFoundError:
    print("Lỗi: Không tìm thấy subject nào. "
          "Hãy kiểm tra lại đường dẫn 'project_dir' của bạn.")
    SUBJECT_IDS = ['01'] # Chỉ chạy test với 'sub-01'

print(f"Sẽ xử lý các subjects: {SUBJECT_IDS}")

# Các thiết lập cho Preprocessing
TASK_NAME = 'oddball'      
FILTER_L_FREQ = 1.0        # Lọc tần số thấp (Hz)
FILTER_H_FREQ = 40.0       # Lọc tần số cao (Hz)
NOTCH_FREQ = 50.0          # Lọc nhiễu điện lưới (50Hz ở VN, 60Hz ở Mỹ)

# === SỬA LỖI RuntimeError (cho sub-15) ===
ICA_COMPONENTS = 0.999     # Tăng ngưỡng để đảm bảo luôn có > 1 component

EOG_CHANNELS = ['Fp1', 'Fp2'] # Kênh EEG trán để phát hiện chớp mắt

# Thư mục BIDS chuẩn để lưu kết quả đã xử lý
derivatives_dir = os.path.join(project_dir, 'derivatives', 'preprocessing')
os.makedirs(derivatives_dir, exist_ok=True)


# =============================================================================
# BƯỚC 1: BẮT ĐẦU VÒNG LẶP QUA TỪNG SUBJECT
# =============================================================================

def run_preprocessing():
    """Hàm chính chạy toàn bộ pipeline."""
    
    # Tắt chế độ tương tác để script chạy tự động
    plt.ion() 

    for subject_id in SUBJECT_IDS:
        
        # === THÊM CƠ CHẾ BỎ QUA (SKIP) NẾU ĐÃ XỬ LÝ ===
        
        # 1.0. Định nghĩa đường dẫn TỆP ĐẦU RA (kết quả)
        output_sub_dir = os.path.join(derivatives_dir, f'sub-{subject_id}', 'eeg')
        output_filename = f'sub-{subject_id}_task-{TASK_NAME}_desc-preproc_eeg.fif'
        output_fpath = os.path.join(output_sub_dir, output_filename)
        
        # 1.1. Kiểm tra xem tệp đầu ra đã tồn tại chưa
        if os.path.exists(output_fpath):
            print(f"\n--- ⏩ Bỏ qua Subject: {subject_id} (tệp {output_filename} đã tồn tại) ---")
            continue  # Bỏ qua subject này, chuyển sang subject tiếp theo
            
        # Nếu chưa tồn tại, bắt đầu xử lý
        print(f"\n--- 🚀 Bắt đầu xử lý Subject: {subject_id} ---")
        os.makedirs(output_sub_dir, exist_ok=True) # Đảm bảo thư mục đầu ra tồn tại
        
        # === KẾT THÚC CƠ CHẾ BỎ QUA ===
        
        
        # 1.2. Định nghĩa đường dẫn BIDS (đầu vào)
        bids_path = BIDSPath(
            subject=subject_id,
            task=TASK_NAME,
            root=project_dir,
            datatype='eeg',
            suffix='eeg',
            extension='.vhdr'
        )
        
        # 1.3. Tải dữ liệu thô (Raw)
        try:
            # Bước 1: Đọc thông tin file (CHƯA tải data vào RAM)
            raw = read_raw_bids(bids_path=bids_path, verbose=False)
            
            # Bước 2: Tải data vào RAM (để chạy được filter ở bước sau)
            raw.load_data(verbose=False)
            
        except FileNotFoundError:
            print(f"*** Lỗi: Không tìm thấy dữ liệu cho {subject_id}. Bỏ qua...")
            continue
            
        # Tải thông tin kênh (tùy chọn nhưng nên có)
        raw.set_montage('standard_1020', on_missing='ignore')

        print(f"Tải thành công: {bids_path.basename}")

        # =====================================================================
        # BƯỚC 2: TIỀN XỬ LÝ (FILTERING)
        # =====================================================================
        
        # 2.1. Lọc nhiễu điện lưới (Notch)
        raw.notch_filter(freqs=NOTCH_FREQ, verbose=False)
        
        # 2.2. Lọc thông dải (Band-pass)
        raw.filter(l_freq=FILTER_L_FREQ, h_freq=FILTER_H_FREQ, verbose=False)
        
        print(f"Đã lọc (Filter & Notch) xong.")

        # =====================================================================
        # BƯỚC 3: LOẠI BỎ NHIỄU CHỚP MẮT (ICA)
        # =====================================================================
        
        # 3.1. Tạo bản sao chỉ để chạy ICA (fit trên data 1Hz)
        ica_raw = raw.copy().filter(l_freq=1.0, h_freq=None, verbose=False)
        
        # 3.2. Thiết lập và Fit ICA
        ica = mne.preprocessing.ICA(
            n_components=ICA_COMPONENTS,
            random_state=97,  
            max_iter='auto',
            verbose=False
        )
        
        # Thêm try...except để bắt lỗi hiếm gặp
        try:
            ica.fit(ica_raw)
        except RuntimeError as e:
            print(f"*** LỖI ICA.FIT thất bại cho sub-{subject_id}: {e}")
            print("     Bỏ qua subject này.")
            del raw, ica_raw
            continue # Bỏ qua subject này

        print("Đã fit ICA xong.")

        # 3.3. Tự động tìm các component chớp mắt (EOG)
        eog_indices, eog_scores = ica.find_bads_eog(
            raw, 
            ch_name=EOG_CHANNELS,
            verbose=False
        )
        
        if not eog_indices:
            print(f"*** Cảnh báo: Không tự động tìm thấy EOG cho sub-{subject_id}."
                   " Có thể cần kiểm tra thủ công.")
            ica.exclude = []
        else:
            print(f"Tìm thấy {len(eog_indices)} EOG component: {eog_indices}")
            ica.exclude = eog_indices

        # =====================================================================
        # BƯỚC 4: TRỰC QUAN HÓA VÀ ĐÁNH GIÁ ICA (Quan trọng)
        # =====================================================================
        
        # --- 4.1. Trực quan hóa KIỂM TRA (Inspection) ---
        # (Đã tắt do lỗi 'overlapping positions' của dataset)
        if ica.exclude:
            print(f"Đã tìm thấy EOG component, bỏ qua bước vẽ biểu đồ do lỗi 'overlapping positions'.")
        else:
            print("Không có EOG component nào được tìm thấy để vẽ biểu đồ.")
        
        
        # --- 4.2. Trực quan hóa ĐÁNH GIÁ (Evaluation) - So sánh TRƯỚC vs SAU ---
        print("Đánh giá hiệu quả: So sánh tín hiệu TRƯỚC và SAU khi lọc ICA...")

        # Tạo epochs (đoạn cắt) xung quanh sự kiện chớp mắt
        eog_epochs_before = mne.preprocessing.create_eog_epochs(
            raw, 
            ch_name=EOG_CHANNELS, 
            verbose=False
        )
        # Bỏ qua nếu không tìm thấy chớp mắt nào
        if len(eog_epochs_before) == 0:
            print(f"*** Cảnh báo: Không tìm thấy sự kiện chớp mắt nào cho sub-{subject_id}.")
            
            # Nếu không có chớp mắt, raw_clean chỉ là bản sao của raw
            raw_clean = ica.apply(raw.copy(), verbose=False)
            
        else:
            # Lấy tín hiệu trung bình của tất cả các lần chớp mắt (TRƯỚC KHI LỌC)
            avg_eog_before = eog_epochs_before.average()

            # Áp dụng ICA để làm sạch dữ liệu
            raw_clean = ica.apply(raw.copy(), verbose=False)

            # Tạo epochs chớp mắt trên dữ liệu ĐÃ LÀM SẠCH
            eog_epochs_after = mne.preprocessing.create_eog_epochs(
                raw_clean, 
                ch_name=EOG_CHANNELS, 
                verbose=False
            )
            
            # Lấy tín hiệu trung bình (SAU KHI LỌC)
            avg_eog_after = eog_epochs_after.average()

            # Vẽ 2 biểu đồ so sánh
            fig, axes = plt.subplots(2, 1, figsize=(10, 8), sharex=True)
            
            # Dữ liệu gốc (nhiễu)
            avg_eog_before.plot(axes=axes[0], show=False)
            axes[0].set_title(f"TRƯỚC KHI LỌC ICA (sub-{subject_id}) - Còn nhiễu chớp mắt")
            
            # Dữ liệu sạch
            avg_eog_after.plot(axes=axes[1], show=False)
            axes[1].set_title(f"SAU KHI LỌC ICA (sub-{subject_id}) - Đã giảm nhiễu")
            
            try:
                fig.tight_layout()
            except UserWarning:
                pass # Bỏ qua cảnh báo 'tight_layout'
            
            # Lưu ảnh so sánh
            fig_path_compare = os.path.join(derivatives_dir, 
                                            f'sub-{subject_id}_task-oddball_ica_compare.png')
            fig.savefig(fig_path_compare, dpi=150, bbox_inches='tight')
            plt.close(fig)
            
            # Giải phóng bộ nhớ cho ảnh
            del avg_eog_before, avg_eog_after, eog_epochs_before, eog_epochs_after


        # =====================================================================
        # BƯỚC 5: LƯU DỮ LIỆU SẠCH (Đã sửa lỗi & đơn giản hóa)
        # =====================================================================
        
        # Đường dẫn (output_fpath) và tên tệp (output_filename) 
        # đã được định nghĩa ở đầu vòng lặp (BƯỚC 1.0)
        
        # 5.4. Lưu tệp .fif
        raw_clean.save(output_fpath, overwrite=True, verbose=False)
        
        print(f"✅ Đã xử lý và lưu thành công! \n   File: {output_filename}")
        
        # Xóa biến để giải phóng RAM cho vòng lặp subject tiếp theo
        del raw, raw_clean, ica_raw, ica

    print("\n--- 🎉 Đã hoàn thành xử lý tất cả subjects! ---")

# =============================================================================
# CHẠY SCRIPT
# =============================================================================
if __name__ == "__main__":
    run_preprocessing()

Sẽ xử lý các subjects: ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42']

--- ⏩ Bỏ qua Subject: 01 (tệp sub-01_task-oddball_desc-preproc_eeg.fif đã tồn tại) ---

--- ⏩ Bỏ qua Subject: 02 (tệp sub-02_task-oddball_desc-preproc_eeg.fif đã tồn tại) ---

--- ⏩ Bỏ qua Subject: 03 (tệp sub-03_task-oddball_desc-preproc_eeg.fif đã tồn tại) ---

--- ⏩ Bỏ qua Subject: 04 (tệp sub-04_task-oddball_desc-preproc_eeg.fif đã tồn tại) ---

--- ⏩ Bỏ qua Subject: 05 (tệp sub-05_task-oddball_desc-preproc_eeg.fif đã tồn tại) ---

--- ⏩ Bỏ qua Subject: 06 (tệp sub-06_task-oddball_desc-preproc_eeg.fif đã tồn tại) ---

--- ⏩ Bỏ qua Subject: 07 (tệp sub-07_task-oddball_desc-preproc_eeg.fif đã tồn tại) ---

--- ⏩ Bỏ qua Subject: 08 (tệp sub-08_task-oddball_desc-preproc_eeg.fif đã tồn tại) ---

--- ⏩ Bỏ qua Subjec

C:\Users\Admin\AppData\Local\Temp\ipykernel_5180\2522065077.py:105: RuntimeWarning: Omitted 748 annotation(s) that were outside data range.
  raw = read_raw_bids(bids_path=bids_path, verbose=False)
C:\Users\Admin\AppData\Local\Temp\ipykernel_5180\2522065077.py:105: RuntimeWarning: Unable to map the following column(s) to to MNE:
medication use: 0
medication type: n/a
caffeine uptake: 0
stress: 3
rest: 4
menstrual cycle: n/a
UMACL/UWIST Niezadowolony Dissatisfied: 4
UMACL/UWIST Energiczny Energetic: 3
UMACL/UWIST Zrelaksowany Relaxed: 3
UMACL/UWIST Bezczynny Idle: 3
UMACL/UWIST Zdenerwowany Nervous: 3
UMACL/UWIST Passive Bierny: 3
UMACL/UWIST Pogodny Bright: 2
UMACL/UWIST Napiety Tense: 2
UMACL/UWIST Stremowany Anxious: 3
UMACL/UWIST Niemrawy Dull: 4
UMACL/UWIST Zmartwiony Stirred up: 3
UMACL/UWIST Zniecierpliowy Impatient: 2
UMACL/UWIST Przygnebiony Depressed: 4
UMACL/UWIST Spiacy Sleepy: 4
UMACL/UWIST Niespokojny Jittery: 3
UMACL/UWIST Usatysfakcjonowany Satisfied: 2
UMACL/UWIST  Bez 

Tải thành công: sub-15_task-oddball_eeg.vhdr
Đã lọc (Filter & Notch) xong.
Fitting ICA to data using 127 channels (please be patient, this may take a while)
Selecting by explained variance: 27 components
Fitting ICA took 73.6s.
Đã fit ICA xong.
Tìm thấy 2 EOG component: [2, 7]
Đã tìm thấy EOG component, bỏ qua bước vẽ biểu đồ do lỗi 'overlapping positions'.
Đánh giá hiệu quả: So sánh tín hiệu TRƯỚC và SAU khi lọc ICA...


C:\Users\Admin\AppData\Local\Temp\ipykernel_5180\2522065077.py:229: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


✅ Đã xử lý và lưu thành công! 
   File: sub-15_task-oddball_desc-preproc_eeg.fif

--- 🚀 Bắt đầu xử lý Subject: 16 ---


...

✅ Đã xử lý và lưu thành công! 
   File: sub-41_task-oddball_desc-preproc_eeg.fif

--- 🚀 Bắt đầu xử lý Subject: 42 ---


C:\Users\Admin\AppData\Local\Temp\ipykernel_5180\2522065077.py:105: RuntimeWarning: Omitted 744 annotation(s) that were outside data range.
  raw = read_raw_bids(bids_path=bids_path, verbose=False)
C:\Users\Admin\AppData\Local\Temp\ipykernel_5180\2522065077.py:105: RuntimeWarning: Unable to map the following column(s) to to MNE:
medication use: 0
medication type: n/a
caffeine uptake: 0
stress: 1
rest: 5
menstrual cycle: n/a
UMACL/UWIST Niezadowolony Dissatisfied: 4
UMACL/UWIST Energiczny Energetic: 1
UMACL/UWIST Zrelaksowany Relaxed: 1
UMACL/UWIST Bezczynny Idle: 4
UMACL/UWIST Zdenerwowany Nervous: 4
UMACL/UWIST Passive Bierny: 4
UMACL/UWIST Pogodny Bright: 1
UMACL/UWIST Napiety Tense: 4
UMACL/UWIST Stremowany Anxious: 4
UMACL/UWIST Niemrawy Dull: 4
UMACL/UWIST Zmartwiony Stirred up: 4
UMACL/UWIST Zniecierpliowy Impatient: 4
UMACL/UWIST Przygnebiony Depressed: 4
UMACL/UWIST Spiacy Sleepy: 4
UMACL/UWIST Niespokojny Jittery: 4
UMACL/UWIST Usatysfakcjonowany Satisfied: 1
UMACL/UWIST  Bez 

Tải thành công: sub-42_task-oddball_eeg.vhdr
Đã lọc (Filter & Notch) xong.
Fitting ICA to data using 127 channels (please be patient, this may take a while)
Selecting by explained variance: 74 components
Fitting ICA took 142.7s.
Đã fit ICA xong.
Tìm thấy 3 EOG component: [0, 1, 3]
Đã tìm thấy EOG component, bỏ qua bước vẽ biểu đồ do lỗi 'overlapping positions'.
Đánh giá hiệu quả: So sánh tín hiệu TRƯỚC và SAU khi lọc ICA...


C:\Users\Admin\AppData\Local\Temp\ipykernel_5180\2522065077.py:229: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  fig.tight_layout()


✅ Đã xử lý và lưu thành công! 
   File: sub-42_task-oddball_desc-preproc_eeg.fif

--- 🎉 Đã hoàn thành xử lý tất cả subjects! ---
